In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import datetime
import tensorflow as tf
import tflearn
import numpy as np
from sklearn.model_selection import train_test_split

import drqn
import student as st

from experience_buffer import ExperienceBuffer
import dataset_utils as d_utils
import utils
import models_dict_utils

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

### Preprocessing Data for DRQN
We take the data from data generator and save them into traces of (s,a,r,sp) tuples.

Each trajectory corresponds to a trace.

If trajectory has length n, then trace will have length n-1. (since we need the next state sp)

In [2]:
data = d_utils.load_data(filename="../synthetic_data/test-n10000-l3-random.pickle")

In [3]:
dqn_data = d_utils.preprocess_data_for_dqn(data, reward_model="dense")

In [4]:
# Single Trace
print (dqn_data[0])

[[array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]), array([ 1.,  0.,  0.,  0.,  0.]), 0.20000000000000001, array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])], [array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), array([ 0.,  0.,  0.,  0.,  1.]), 0.20000000000000001, array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.])]]


In [5]:
# First tuple in a trace
s,a,r,sp = dqn_data[0][0]
print (s)
print (a)
print (r)
print (sp)

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
[ 1.  0.  0.  0.  0.]
0.2
[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [6]:
# Last tuple
s,a,r,sp = dqn_data[0][-1]
print (s)
print (a)
print (r)
print (sp)

[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  1.]
0.2
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]


In [7]:
dqn_data_train, dqn_data_test = train_test_split(dqn_data, test_size=0.2)

### Creating a DRQN model and training it

In [8]:
model_id = "test_model_drqn"

# Directory for storing tensorboard summaries
tensorboard_dir = '../tensorboard_logs/' + model_id + '/'
summary_interval = 100
checkpoint_dir = '../checkpoints/' + model_id + '/'
checkpoint_path = checkpoint_dir + '_/'

utils.check_if_path_exists_or_create(tensorboard_dir)
utils.check_if_path_exists_or_create(checkpoint_dir)
    
checkpoint_interval = 200

In [9]:
# Create the model object
model = drqn.DRQNModel(model_id, timesteps=2)

In [10]:
# Initialize trainer object inside the model
model.init_trainer(tensorboard_dir=tensorboard_dir, save_ckpt_path=checkpoint_path)

In [11]:
# Creating training and validation data
train_buffer = ExperienceBuffer()
train_buffer.buffer = dqn_data_train
train_buffer.buffer_sz = len(train_buffer.buffer)

val_buffer = ExperienceBuffer()
val_buffer.buffer = dqn_data_test
val_buffer.buffer_sz = len(val_buffer.buffer)

date_time_string = datetime.datetime.now().strftime("%m-%d-%Y_%H-%M-%S")
run_id = "{}".format(date_time_string)

In [12]:
# train the model (uses the previously initialized trainer object)
date_time_string = datetime.datetime.now().strftime("%m-%d-%Y_%H-%M-%S")
run_id = "{}".format(date_time_string)
model.train(train_buffer, val_buffer, n_epoch=2,
              run_id=run_id, load_checkpoint=True,load_ckpt_path=checkpoint_dir)

Training Step: 38624  | total loss: 0.08976 | time: 2.187s
| Optimizer | epoch: 002 | loss: 0.08976 -- iter: 7936/8000
Training Step: 38625  | total loss: 0.08939 | time: 3.208s
| Optimizer | epoch: 002 | loss: 0.08939 | val_loss: 0.08639 -- iter: 8000/8000
--
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'


In [13]:
# init evaluator of the model
model.init_evaluator(load_ckpt_path=checkpoint_dir)

In [14]:
# Create inputs (states / observations so far) to use for predictions
from drqn import stack_batch
train_batch = train_buffer.sample_in_order(4)

# make sure that batches are over multiple timesteps, should be of shape (batch_sz, n_timesteps, ?)
s_batch_train = stack_batch(train_batch[:, :, 0])  # current states

In [15]:
# Use model to predict next action
actions, q_vals = model.predict(s_batch_train, last_timestep_only=True)

1


In [16]:
q_vals

array([[ 0.30812424,  0.30810687,  0.30813327,  0.30798167,  0.30711538],
       [ 0.43627185,  0.43623003,  0.43638146,  0.43616685,  0.43632478],
       [ 0.43779975,  0.43772992,  0.4377695 ,  0.43766207,  0.43770874],
       [ 0.39733657,  0.39734417,  0.39731672,  0.39723107,  0.39730671]])

In [17]:
actions

array([2, 2, 0, 1])